# Antibody landscapes after influenza virus infection or vaccination
 in the next notbook ther is implemntation of Antibody landscapes after influenza virus infection or vaccination article by J. M. Fonville et al
 

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tools import *
from matplotlib.lines import Line2D


In [3]:
# ==== USER PARAMETERS ====

SAMPLE = (11101, 'D-28 predose') # sample number of the data
SAMPLE_2=   (11101, 'Follow up') #optional : when we want to compare 2 measurment of 2 patients or patient before and after
GRID_SIZE = 200        # Grid resolution for plotting or interpolation
A = 11              # Distance threshold to define a "distant" antigen
CLOSE_NEIGH = 5     # Radius threshold for regression surface - points which far form this point will not be clacualted
NORMALIZE = True       # Whether to normalize the X and Y coordinates
SMOOTH_FACTOR =50    # Spline smoothing factor for the summary path
OPTIMIZATION=False
DATA_DIR='patients_bt_years.csv'
FLUGEN=True
SAVE_PATH=f'output/figures/patient_{SAMPLE[0]}_{SAMPLE[1]}/'

### 1. Data processing

### 1.1 Import Data


In [ ]:
df=pd.read_excel("flugen_lilach/flugen_002_sera_IgA_for_Gal.xlsx",index_col='ptid')
df

In [5]:

df_antigenic_map=pd.read_csv("antigentic_map.csv")
df_antigenic_map_by_gal=pd.read_csv("antigen_map/antigen_coordinates.csv")
df=pd.read_csv(DATA_DIR)
df=df.iloc[:,2:]
#fluegen
if FLUGEN:
    #data preprocessing
    df=pd.read_excel("flugen_lilach/flugen_002_sera_IgA_for_Gal.xlsx",index_col='ptid')
    df=df.groupby(by=['ptid', 'timepoint']).mean(numeric_only=True)

    hertz_titer=pd.read_csv("fluegen/list_of_identical_seqs.csv").iloc[:-1,:]
    rename_dict = dict(zip(hertz_titer['name'], hertz_titer['similiar_seq_name']))
    df=df[list(rename_dict.keys())].rename(columns=rename_dict)



In [ ]:
##  for each patient 3 grpah of d1-d28-followup
# do it for placebo and m2sr
#do it fo all patients
df

### 1.2 Pre-preocess Data

1) To generate the antibody landscape, titers were converted onto a log scale : z= log2(titer/10)
2) undetectable titer ("<10") are converted to 5
3) for the regrssion model- data was normlized by MINMAX scaler by the x and y axis of the antigneic map


In [ ]:
#preprocess- 
df=df.applymap(clean_titer)
df=df.applymap(convert_titer)
unique_clusters = df_antigenic_map['cluster'].unique()

    

#patient1 preprocess
sample1=df.transpose()[SAMPLE]
sample1=sample1.rename("log_titers")
sample1 = sample1.dropna()  # drop missing titers just in case


# Merge coordinates and titers for a single ferret sample
# Merge antigenic coordinates and titer data
df_antigenic_map["color"]=df_antigenic_map['cluster'].map(color_map)
df_landscape = df_antigenic_map.merge(sample1, left_on='viruses', right_index=True)

# scaler = MinMaxScaler()
# df_landscape['AG coordinate 1']=scaler.fit_transform(df_landscape['AG coordinate 1'].values.reshape(-1, 1)).flatten()
# df_landscape['AG coordinate 2']=scaler.fit_transform(df_landscape['AG coordinate 2'].values.reshape(-1, 1)).flatten()

# Extract coordinates and colors
X = df_landscape['AG coordinate 2'].values
Y = df_landscape['AG coordinate 1'].values
Z = df_landscape['log_titers'].values
C = df_landscape['color']
cluster_label1=df_landscape["cluster"].values

if SAMPLE_2:
    #patient2 preprocess
    sample2=df.transpose()[SAMPLE_2]
    sample2=sample2.rename("log_titers")
    sample2 = sample2.dropna()  # drop missing titers just in case

    #fliter only antigen which has shown in sample1
    sample2 = sample2[sample2.index.intersection(sample1.index)]
    
    # Merge coordinates and titers for a single ferret sample
    # Merge antigenic coordinates and titer data
    df_landscape_2 = df_antigenic_map.merge(sample2, left_on='viruses', right_index=True)
    df_landscape_2['color'] = df_landscape_2['cluster'].map(color_map)

    # scaler = MinMaxScaler()
    # df_landscape_2['AG coordinate 1']=scaler.fit_transform(df_landscape_2['AG coordinate 1'].values.reshape(-1, 1)).flatten()
    # df_landscape_2['AG coordinate 2']=scaler.fit_transform(df_landscape_2['AG coordinate 2'].values.reshape(-1, 1)).flatten()

    # Extract coordinates and colors
    X_2 = df_landscape_2['AG coordinate 2'].values
    Y_2 = df_landscape_2['AG coordinate 1'].values
    Z_2 = df_landscape_2['log_titers'].values
    C_2 = df_landscape_2['color']
    cluster_label2=df_landscape_2["cluster"].values

### 2. Antigenic Map

Antigenic cartography positions sera and viruses in a map based on their HI titrations such that distance between
viruses represents their antigenic relationship

In [ ]:
plot_antigenic_map(df_antigenic_map,color_map,unique_clusters)

In [ ]:

# After your existing plotting code, get the current Axes:
ax = plt.gca()

# Build legend handles and labels
handles = []
labels  = []

rename_dict_transpose = {v: k for k, v in rename_dict.items()}
plot_legend=df_landscape[['viruses','color']]
plot_legend["viruses"]=plot_legend["viruses"].apply(lambda x : rename_dict_transpose[x])
for vid, color in zip(plot_legend['viruses'], plot_legend['color']):
    display_name = vid
    handles.append(Line2D(
        [0], [0],
        marker='o',
        color='w',
        markerfacecolor=color,
        markersize=8
    ))
    labels.append(display_name)

# Place the legend just outside the plot on the right
ax.legend(
    handles, labels,
    title="Strains",
    loc='upper left',
    bbox_to_anchor=(1.02, 1),
    borderaxespad=0.,
    frameon=False
)

plt.tight_layout()
plt.show()

### 3.Antibody Landscape


#### 3.1: 3d Antibody Landscape: 
3D represntation of the titer for each antigen for each patient

##### Sample 1

In [ ]:
plot_3d_antibody(X,Y,Z,C,SAVE_PATH)

##### Sample 2 (Optional)

In [ ]:
plot_3d_antibody(X_2,Y_2,Z_2,C_2)

### 3.2 Antibody summary path 

Here, we compute a path that passes through the different antigenic clusters. We then plot this path in a 2D visualization, where the X-axis represents the summary path and the Y-axis represents the Titer. A smoothing spline is applied to trace a curve that passes through the different antigenic clusters

In [ ]:
unique_clusters

In [ ]:
x_smooth,y_smooth,df_centroids=find_summary_path(df_antigenic_map,SMOOTH_FACTOR)

### Sample 2 (Optional)

In [ ]:
x_smooth2,y_smooth2,df_centroids2=find_summary_path(df_antigenic_map,SMOOTH_FACTOR)

## 3.3  Antibody Surface


The algorithm for generating the 3D antibody surface map is based on locally weighted linear regression. The main steps are: <br>

	1.	Initialize the Surface Grid  

Create a 2D grid of size N × N, where N is a hyperparameter called GRID_SIZE. <br>

	2.	Compute Antigenic Distances

For each position on the surface, compute the Euclidean distance to each antigen:
$$
a_{ij} = \sqrt{(x_j - x_i)^2 + (y_j - y_i)^2}
$$
If a distance exceeds a threshold value A, its weight is set to zero. Otherwise, use the tricubic weighting function:
$$
w_{ij} =
\begin{cases}
\left(1 - \left(\frac{a_{ij}}{A}\right)^3\right)^3 & \text{if } a_{ij} \leq A \
0 & \text{if } a_{ij} > A
\end{cases}
$$
	3.	Apply Weighted Linear Regression
At each surface position p_j, perform a locally-weighted linear regression using the titers z_i and coordinates x_i, y_i. The regression is defined as:
$$
z_i = c_j + x_i y_i \beta_j + \epsilon_i
$$
where c_j and \beta_j are regression coefficients specific to each location, optimized by minimizing the weighted sum of squared errors:
$$
S = \sum_{i=1}^{n} w_{ij} \cdot \epsilon_i^2
$$
	4.	Predict Titer Values
The predicted titer for each surface position is computed as:
$$
\hat{z}_j = c_j + x_j y_j \beta_j
$$

Each point on the 3D antibody landscape is defined by three coordinates:

- **\( xi \)**: the x-coordinate on the antigenic map  
- **\( yi \)**: the y-coordinate on the antigenic map  
- **\( zi \)**: the log-transformed HI titer, representing antibody strength

#### Sample 1 

In [ ]:
x_range,y_range,landscape_z,predicted_z =compute_surface_regression(X,Y,Z,GRID_SIZE,A,CLOSE_NEIGH,cluster_label1,unique_clusters,OPTIMIZATION)


In [ ]:
landscape_z

In [ ]:
masked_z,x_path_valid,y_path_valid,z_path_valid=plot3d_antibody_surface(X,Y,Z,C,GRID_SIZE,landscape_z,x_range,y_range,x_smooth,y_smooth,unique_clusters,cluster_label1)

#### Patient 2 (Optional)

In [ ]:
X_2

In [ ]:
x_range,y_range,landscape_z2,predicted_z2 =compute_surface_regression(X_2,Y_2,Z_2,GRID_SIZE,A,CLOSE_NEIGH,cluster_label2,unique_clusters,OPTIMIZATION)


In [ ]:
masked_z2,x2_path_valid,y2_path_valid,z2_path_valid=plot3d_antibody_surface(X_2,Y_2,Z_2,C,GRID_SIZE,landscape_z2,x_range,y_range,x_smooth2,y_smooth2,unique_clusters,cluster_label2)

### 4. 2D antibody landscape

#### Patient 1

In [ ]:
plot_2d_landscape(x_path_valid,y_path_valid,z_path_valid,df_landscape,weight=0.9,y_lim=10,save_path=SAVE_PATH)

#### Patinet 2 (Optional)

In [ ]:
plot_2d_landscape(x2_path_valid,y2_path_valid,z2_path_valid,df_landscape_2,weight=0.9,y_lim=10)

#### Compate patients

In [ ]:
plot_ema_smooth_landscape_with_clusters(x_path_valid, y_path_valid, z_path_valid, z2_path_valid, df_landscape,color_map=color_map ,label_pre=SAMPLE[1], label_post=SAMPLE_2[1],y_lim=10,save_path=SAVE_PATH)

In [ ]:
df_landscape

In [ ]:
df_landscape_2

In [ ]:
df.index.get_level_values(0).unique()

In [ ]:
df

In [ ]:
for ptid in df.index.get_level_values(0).unique():


    try:
        #define parameters
        SAMPLE = (ptid, 'D-28 predose') # sample number of the data
        SAMPLE_2=   (ptid, 'D1 predose') #optional : when we want to compare 2 measurment of 2 patients or patient before and after
        GRID_SIZE = 200        # Grid resolution for plotting or interpolation
        A = 11              # Distance threshold to define a "distant" antigen
        CLOSE_NEIGH = 5     # Radius threshold for regression surface - points which far form this point will not be clacualted
        NORMALIZE = True       # Whether to normalize the X and Y coordinates
        SMOOTH_FACTOR =50    # Spline smoothing factor for the summary path
        OPTIMIZATION=False
        DATA_DIR='patients_bt_years.csv'
        FLUGEN=True
        SAVE_PATH=f'output/figures/patient_{SAMPLE[0]}_{SAMPLE[1]}/'

        
        #preprocess- 


        #patient1 preprocess
        sample1=df.transpose()[SAMPLE]
        sample1=sample1.rename("log_titers")
        sample1 = sample1.dropna()  # drop missing titers just in case


        # Merge coordinates and titers for a single ferret sample
        # Merge antigenic coordinates and titer data
        df_antigenic_map["color"]=df_antigenic_map['cluster'].map(color_map)
        df_landscape = df_antigenic_map.merge(sample1, left_on='viruses', right_index=True)

        # scaler = MinMaxScaler()
        # df_landscape['AG coordinate 1']=scaler.fit_transform(df_landscape['AG coordinate 1'].values.reshape(-1, 1)).flatten()
        # df_landscape['AG coordinate 2']=scaler.fit_transform(df_landscape['AG coordinate 2'].values.reshape(-1, 1)).flatten()

        # Extract coordinates and colors
        X = df_landscape['AG coordinate 2'].values
        Y = df_landscape['AG coordinate 1'].values
        Z = df_landscape['log_titers'].values
        C = df_landscape['color']
        cluster_label1=df_landscape["cluster"].values

        if SAMPLE_2:
            #patient2 preprocess
            sample2=df.transpose()[SAMPLE_2]
            sample2=sample2.rename("log_titers")
            sample2 = sample2.dropna()  # drop missing titers just in case

            #fliter only antigen which has shown in sample1
            sample2 = sample2[sample2.index.intersection(sample1.index)]
            
            # Merge coordinates and titers for a single ferret sample
            # Merge antigenic coordinates and titer data
            df_landscape_2 = df_antigenic_map.merge(sample2, left_on='viruses', right_index=True)
            df_landscape_2['color'] = df_landscape_2['cluster'].map(color_map)

            # scaler = MinMaxScaler()
            # df_landscape_2['AG coordinate 1']=scaler.fit_transform(df_landscape_2['AG coordinate 1'].values.reshape(-1, 1)).flatten()
            # df_landscape_2['AG coordinate 2']=scaler.fit_transform(df_landscape_2['AG coordinate 2'].values.reshape(-1, 1)).flatten()

            # Extract coordinates and colors
            X_2 = df_landscape_2['AG coordinate 2'].values
            Y_2 = df_landscape_2['AG coordinate 1'].values
            Z_2 = df_landscape_2['log_titers'].values
            C_2 = df_landscape_2['color']
            cluster_label2=df_landscape_2["cluster"].values


            #run code for all data
            plot_3d_antibody(X,Y,Z,C,SAVE_PATH)
            plot_3d_antibody(X_2,Y_2,Z_2,C_2,SAVE_PATH,SAMPLE2=True)
            x_smooth,y_smooth,df_centroids=find_summary_path(df_antigenic_map,SMOOTH_FACTOR)
            x_smooth2,y_smooth2,df_centroids2=find_summary_path(df_antigenic_map,SMOOTH_FACTOR)
            
            #logistic regression for 1
            x_range,y_range,landscape_z,predicted_z =compute_surface_regression(X,Y,Z,GRID_SIZE,A,CLOSE_NEIGH,cluster_label1,unique_clusters,OPTIMIZATION)
            masked_z,x_path_valid,y_path_valid,z_path_valid=plot3d_antibody_surface(X,Y,Z,C,GRID_SIZE,landscape_z,x_range,y_range,x_smooth,y_smooth,unique_clusters,cluster_label1)


            #logistic regression for 2
            x_range,y_range,landscape_z2,predicted_z2 =compute_surface_regression(X_2,Y_2,Z_2,GRID_SIZE,A,CLOSE_NEIGH,cluster_label2,unique_clusters,OPTIMIZATION)
            masked_z2,x2_path_valid,y2_path_valid,z2_path_valid=plot3d_antibody_surface(X_2,Y_2,Z_2,C,GRID_SIZE,landscape_z2,x_range,y_range,x_smooth2,y_smooth2,unique_clusters,cluster_label2)


            #plot 
            plot_2d_landscape(x_path_valid,y_path_valid,z_path_valid,df_landscape,weight=0.9,y_lim=10,save_path=SAVE_PATH)
            plot_ema_smooth_landscape_with_clusters(x_path_valid, y_path_valid, z_path_valid, z2_path_valid, df_landscape,color_map=color_map ,label_pre=SAMPLE[1], label_post=SAMPLE_2[1],y_lim=10,save_path=SAVE_PATH)
    except Exception as e:
        print(f"Error processing patient {ptid}: {e}")
        continue

In [ ]:
f